In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import mysql.connector
import re

In [8]:
column_names = ['title','url','date']
new_df = pd.DataFrame(columns = column_names)

In [9]:
idx = 0

In [19]:
chromedriver_path = "C:/Users/kkksk/.wdm/drivers/chromedriver/win64/125.0.6422.78/chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [20]:
# cur_month = 7
cur_year = 2019
target_year = 2023

while cur_year <= target_year:
    
    url = f"https://www.google.com/search?q=oil&tbs=cdr:1,cd_min:1/1/{cur_year},cd_max:12/31/{cur_year}&tbm=nws"
    driver.get(url)
    
    time.sleep(30)
    
    for _ in range(30):
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso > div >div>div>div')
        for news_div in news_results:
            try:
                news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                news_text_elements = news_div.find_elements(By.CSS_SELECTOR, 'a>div>div>div')
                news_item = [elem.text for elem in news_text_elements]
                if len(news_item) >= 5:
                    Link = news_link
                    Title = news_item[2]
                    Date = news_item[4]
                
                new_df.loc[idx] = [Title, Link, Date]
                idx += 1
            except Exception as e:
                print(e)
        
        try:
            next_button = driver.find_element(By.ID, 'pnnext')
            next_button.click()
            time.sleep(1)
        except NoSuchElementException:
            break
    # if cur_month == 7:  
    #     cur_month = 1
    cur_year += 1
    # else: 
    #     cur_month += 6


In [18]:
new_df

,title,url,date
0,Riyadh Crude Oil-Fired Power Plant,https://www.power-technology.com/projects/riyadh/,2002. 1. 29.
1,Germans Invent Super Sponge for Oil Spill Cleanup,https://www.dw.com/en/germans-invent-super-spo...,2002. 11. 25.
2,Palm oil: Alternative lipid source in aquacult...,https://www.globalseafood.org/advocate/palm-oi...,2002. 4. 1.
3,Tanker spills oil on Spanish coast,https://www.nature.com/articles/news021111-10,2002. 11. 14.
4,Barracuda and Caratinga Fields,https://www.offshore-technology.com/projects/b...,2002. 3. 4.
...,...,...,...
4539,"'We cannot swim, we cannot eat': Solomon Islan...",https://www.theguardian.com/environment/2019/m...,2019. 3. 6.
4540,"Lemongrass essential oil: Benefits, use, and s...",https://www.medicalnewstoday.com/articles/325209,2019. 5. 18.
4541,Scientists extract hydrogen gas from oil and b...,https://phys.org/news/2019-08-scientists-hydro...,2019. 8. 19.
4542,"Alberta, Canada's oil sands is the world's mos...",https://www.nationalgeographic.com/environment...,2019. 4. 11.


In [14]:
new_df.to_parquet('news_oil_02~23.parquet', engine = 'pyarrow', compression = 'gzip')